In [1]:
import numpy
from collections import Counter
from keras.preprocessing.sequence import pad_sequences # 引入序列模型的pad
import pickle #序列化模块
import platform
from keras.models import Sequential #引入序列模型
from keras.layers import Embedding, Bidirectional, LSTM # 用到的BILSTM 
from keras_contrib.layers import CRF #引入CRF模块

Using TensorFlow backend.


In [2]:
def load_data():
    train = _parse_data(open('data/train_data.txt', 'rb'))
    test = _parse_data(open('data/test_data.txt', 'rb'))
    # 构建词映射
    word_counts = Counter(row[0].lower() for sample in train for row in sample)
    vocab = [w for w, f in iter(word_counts.items()) if f >= 1]
    chunk_tags = ['B-K'
    ,'E-K'
    ,'I-K'
    ,'O']

    train = _process_data(train, vocab, chunk_tags)
    test = _process_data(test, vocab, chunk_tags)
    return train, test, (vocab, chunk_tags)


def _parse_data(fh):
    # 预处理文本
    split_text = '\n' #去除回车符
    string = fh.read().decode('utf-8') 
    data = [[row.split() for row in sample.split(split_text)] \
            for sample in string.strip().split(split_text + split_text)]
    fh.close()
    return data


def _process_data(data, vocab, chunk_tags, maxlen=None):
    if maxlen is None:
        maxlen = max(len(s) for s in data)
    word2idx = dict((w, i) for i, w in enumerate(vocab))
    x = [[word2idx.get(w[0].lower(), 1) for w in s] for s in data]  

    y_chunk = [[chunk_tags.index(w[1]) for w in s] for s in data]

    x = pad_sequences(x, maxlen)  # left padding

    y_chunk = pad_sequences(y_chunk, maxlen, value=-1)

    y_chunk = numpy.expand_dims(y_chunk, 2)
    
    return x, y_chunk


def process_data(data, vocab, maxlen=100):
    word2idx = dict((w, i) for i, w in enumerate(vocab))
    x = [word2idx.get(w[0].lower(), 1) for w in data]
    length = len(x)
    x = pad_sequences([x], maxlen)  # left padding
    return x, length

In [3]:
train = _parse_data(open('data/train_data.txt', 'rb'))
test = _parse_data(open('data/test_data.txt', 'rb'))

In [16]:
EMBED_DIM = 20 # 嵌入层的维度
BiRNN_UNITS = 100 # BILSTM的unit

In [17]:
(train_x, train_y), (test_x, test_y), (vocab, chunk_tags) = load_data()

In [18]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [19]:
model = Sequential() #初始化序列模型
model.add(Embedding(len(vocab), EMBED_DIM, mask_zero=True))  # 输入层的embedding
model.add(Bidirectional(LSTM(BiRNN_UNITS, return_sequences=True))) #双向LSTM
crf = CRF(len(chunk_tags), sparse_target=True,name='crff') # CRF层
model.add(crf)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 20)          13300     
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 200)         96800     
_________________________________________________________________
crff (CRF)                   (None, None, 4)           828       
Total params: 110,928
Trainable params: 110,928
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile('adam', loss=crf.loss_function, metrics=[crf.accuracy]) #模型编译 

In [21]:
model.fit(train_x, train_y,batch_size=16,epochs=15, validation_data=[test_x, test_y]) #训练模型

Train on 556 samples, validate on 166 samples
Epoch 1/15
556/556 [==============================] - 7s 13ms/step - loss: 1.9111 - acc: 0.5890 - val_loss: 1.4169 - val_acc: 0.7992
Epoch 2/15
556/556 [==============================] - 3s 6ms/step - loss: 1.5518 - acc: 0.7371 - val_loss: 1.3615 - val_acc: 0.8003
Epoch 3/15
556/556 [==============================] - 4s 7ms/step - loss: 1.4721 - acc: 0.7835 - val_loss: 1.3148 - val_acc: 0.8343
Epoch 4/15
556/556 [==============================] - 4s 7ms/step - loss: 1.3561 - acc: 0.8545 - val_loss: 1.2492 - val_acc: 0.8535
Epoch 5/15
556/556 [==============================] - 4s 7ms/step - loss: 1.2443 - acc: 0.9172 - val_loss: 1.1630 - val_acc: 0.9527
Epoch 6/15
556/556 [==============================] - 4s 7ms/step - loss: 1.1872 - acc: 0.9562 - val_loss: 1.1102 - val_acc: 0.9695
Epoch 7/15
556/556 [==============================] - 4s 7ms/step - loss: 1.1468 - acc: 0.9666 - val_loss: 1.0865 - val_acc: 0.9771
Epoch 8/15
556/556 [=========

In [22]:
with open('model/config.pkl', 'wb') as outp: # 存词典和标签体系
    pickle.dump((vocab, chunk_tags), outp)

In [23]:
model.save('model/crf_company.h5') #保存模型

In [ ]:
# 北京美丽屋房产经纪有限公司 -->  美丽屋
# 1. 正则匹配+字典：
# 2. HMM 
# 3. tf-idf
# 4. LR 
# 5. BiLSTM+CRF
# 6. CRF

In [4]:
??CRF